In [6]:
import plotly.express as px
import sys
import os
import pandas as pd
import psycopg2
from psycopg2.extras import RealDictCursor

sys.path.append('../Backend/Conversions')
import convertt

sys.path.append('../PredictionModel')
import predict_temp
import predict_rain


In [3]:
def get_dataset(suburbList, years = 1):
    clusterid,datetime,temperature,rainfall = convertt.suburb_to_var(suburbList)
    dataset = {
    'clusterid' : clusterid,
    'datetime': datetime,
    'temperature': temperature,
    'rainfall': rainfall
    }
    dataset = pd.DataFrame(dataset)
    return dataset


In [8]:
suburbs = pd.read_csv("../Datasets/SuburbClustered.csv", index_col=[0])

In [ ]:
-36.022, 147.124

In [9]:
suburbs

,officialnamesuburb,officialnamestate,officialcodelocalgovernmentarea,officialcodestate,lattitude,longtitude,geoshape,clusterid
0,Adaminaby,New South Wales,17040,1,-36.011932,148.786320,"{""coordinates"": [[[148.71675360000006, -36.060...",100412
1,Albury,New South Wales,10050,1,-36.073698,146.913468,"{""coordinates"": [[[146.92431042300007, -36.086...",101407
2,Alectown,New South Wales,16200,1,-32.914255,148.268021,"{""coordinates"": [[[148.19998799900006, -32.966...",92411
3,Alpine,New South Wales,18350,1,-34.408694,150.536771,"{""coordinates"": [[[150.5122384670001, -34.4223...",96417
4,Argoon,New South Wales,15560,1,-34.917979,145.657248,"{""coordinates"": [[[145.66848513900004, -35.017...",97404
...,...,...,...,...,...,...,...,...
15328,Beard,Australian Capital Territory,89399,8,-35.341729,149.210530,"{""coordinates"": [[[149.21876130200005, -35.341...",99414
15329,Bruce (ACT),Australian Capital Territory,89399,8,-35.245480,149.091552,"{""coordinates"": [[[149.09199952500012, -35.234...",98413
15330,Chisholm (ACT),Australian Capital Territory,89399,8,-35.421965,149.124912,"{""coordinates"": [[[149.114427619, -35.41629143...",99413
15331,City,Australian Capital Territory,89399,8,-35.281329,149.129132,"{""coordinates"": [[[149.12790347300006, -35.286...",98413


In [17]:
def coordinate_to_id(latitude, longitude):
    """
    Function to get clusterid given coordinates. Find the the cluster that is closest to the input coordinates

    Input:
        latitude:
        longitude:

    Output:
        clusterid
    """
    conn = psycopg2.connect(
    dbname="climatepulse",
    host="postgres-1.c96iysms626t.ap-southeast-2.rds.amazonaws.com",
    port=5432,
    user="postgres",
    password="Climatepulse123."
    )
    # SQL query to find the closest cluster_id using the Haversine formula
    query = """
    SELECT clusterid, lattitude, longtitude,
           ST_Distance(
               ST_SetSRID(ST_Point(longtitude, lattitude), 4326),  -- Cluster coordinates
               ST_SetSRID(ST_Point(%s, %s), 4326)               -- Input coordinates
           ) AS distance
    FROM suburbclustered
    ORDER BY distance
    LIMIT 1;
    """
    
    try:
        # Execute the query with the provided latitude and longitude
        with conn.cursor(cursor_factory=RealDictCursor) as cursor:
            cursor.execute(query, (longitude, latitude))  # Longitude comes first in ST_Point
            result = cursor.fetchone()
            
            # Check if a result was found
            if result:
                cluster_id = result['clusterid']
                return cluster_id
            else:
                return None  # No cluster found

    except Exception as e:
        print(f"Error occurred: {e}")
        return None
    
    finally:
        conn.close()

In [14]:
r = coordinate_to_id(-36.022, 147.124)

In [16]:
r

100408